In [99]:
import numpy as np
import json
from PIL import Image
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random
from efficientnet_pytorch import EfficientNet
import torchvision
import tensorflow as tf
from torchvision import transforms, datasets

class_names = {
    "0": "circle",      # "0": "원형"
    "1": "hexagon",   # "1": "육각형"
    "2": "octagon",   # "2": "팔각형"
    "3": "oval",  # "3": "타원형"
    "4": "pentagon",   # "4": "오각형"
    "5": "rectangle",   # "5": "직사각형"
    "6": "rhombus",   # "6": "마름모형"
    "7": "semicircular",   # "7": "반원형"
    "8": "square",   # "8": "정사각형"
    "9": "triangle"   # "9": "삼각형"
}

model_name = 'efficientnet-b0'  # b5
image_size = EfficientNet.get_image_size(model_name)
print(image_size)
model = EfficientNet.from_pretrained(model_name, num_classes=10)
PATH = "C:/python/pill_model.pt"
device = torch.device('cpu')
model.load_state_dict(torch.load(PATH, map_location=device))

## make dataset
from torchvision import transforms, datasets
transform = transforms.Compose([
                                    transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                ])

import urllib.request as req
url = 'https://storage.googleapis.com/ocrtestttt/KakaoTalk_20210303_152346768.jpg'

save_path = "C:/python/test/temp.jpg"

image_read = req.urlopen(url).read()
image_open = open(save_path,'wb')
image_open.write(image_read)

img = PIL.Image.open(save_path)
img = data_transforms(img)
model.eval()
prediction = model(img.unsqueeze(0))
print(prediction)
a = torch.argmax(prediction)
temp = str(a.item())
print(class_names[temp])

224
Loaded pretrained weights for efficientnet-b0
tensor([[-3.2135,  0.3781,  1.5526,  0.5304, -1.8650,  5.6038, -1.4492, -1.8365,
          3.2470, -2.8318]], grad_fn=<AddmmBackward>)
rectangle
